In [ ]:
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [1]:
import os
os.chdir('/content/drive/MyDrive/text-detection/')

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

from google.colab.patches import cv2_imshow
from PIL import Image

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode

from utils import *

In [3]:
import cv2
import numpy as np
import string
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional

from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax

from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.activations import elu
from tqdm import tqdm
from collections import Counter

from PIL import Image, ImageFont, ImageDraw, ImageFilter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from IPython.display import clear_output
import random

In [4]:
act_model,char_list,_,_=image_text_model()
act_model.load_weights('models/image_to_text.hdf5')

In [5]:
MAX_OUTPUT_CHARS = 30
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '-PAD-'

eng_alpha2index = {pad_char: 0}
for index, alpha in enumerate(eng_alphabets):
    eng_alpha2index[alpha] = index+1

kannada_alphabets = [chr(alpha) for alpha in range(3202, 3311)]
kannada_alphabet_size = len(kannada_alphabets)

kannada_alpha2index = {pad_char: 0}
for index, alpha in enumerate(kannada_alphabets):
    kannada_alpha2index[alpha] = index+1

translation_model = Transliteration_EncoderDecoder_Attention(len(eng_alpha2index), 256, len(kannada_alpha2index), verbose=True)
translation_model=torch.load('models/translation.pt')
translation_model.eval()

Transliteration_EncoderDecoder_Attention(
  (encoder_rnn_cell): GRU(27, 256)
  (encoder_rnn_cell2): GRU(256, 256)
  (decoder_rnn_cell): GRU(512, 256)
  (decoder_rnn_cell2): GRU(512, 256)
  (h2o): Linear(in_features=256, out_features=110, bias=True)
  (softmax): LogSoftmax(dim=2)
  (U): Linear(in_features=256, out_features=256, bias=True)
  (W): Linear(in_features=256, out_features=256, bias=True)
  (attn): Linear(in_features=256, out_features=1, bias=True)
  (out2hidden): Linear(in_features=110, out_features=256, bias=True)
)

In [148]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 # Set threshold for this model
cfg.MODEL.WEIGHTS = "models/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
predictor = DefaultPredictor(cfg)

In [188]:
def complete_prediction(img_path,font_size=60):
  test_image = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
  outputs = predictor(test_image)
  font_size=font_size
  bounding_box=outputs['instances'][outputs['instances'].pred_classes==1].pred_boxes.tensor.cpu().numpy()
  test_image=Image.open(os.path.join(img_path))
  final_img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
  final_img = Image.fromarray(final_img)
  i=0
  for box in bounding_box:
      crop_img=test_image.crop(box)
      img_path='croppedimages/'+'img'+str(i)+'.jpg'
      crop_img.save(img_path)
      crop_img=pre_process_image(img_path)
      test_word=predict_output(crop_img,act_model,char_list)
      test_word=test_word.upper()
      translated_word=language_translation(translation_model, test_word, eng_alpha2index, device = 'cpu')
      ImageDraw.Draw(final_img).text((box[0], box[1]-70), translated_word, font=unicode_font, fill=(b,g,r))
      i=i+1
  return final_img

In [183]:
unicode_font = ImageFont.truetype("./akshar.ttf", font_size)
b,g,r = 0,0,0

In [190]:
plt.figure(figsize=(10,10))
plt.axis("off")
img_path='/content/new4.jpg'
output_img=complete_prediction(img_path,80)
plt.imshow(output_img);